In [1]:
import datetime
import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

#import emotion_model
import dwdii_bc_model_helper as bc
import bc_models as models

random.seed(20275)

Using Theano backend.


In [2]:
print "device:", theano.config.device
print "floatX:",  theano.config.floatX
print "mode:", theano.config.mode
print "openmp:", theano.config.openmp
print "allow_gc:", theano.config.allow_gc

device: cpu
floatX: float32
mode: FAST_RUN
openmp: True
allow_gc: False


In [3]:
imagePath = "/root/bc_data/ddsm-png.25"
dataPath = "/root/bc_data/ddsm-png.25/Ddsm_png.csv"
imgResize = (150, 150)

In [4]:
os.listdir('/root/bc_data')

['ddsm', 'ddsm-hog', 'ddsm-png.25', 'ddsm-sm', 'Ddsm.csv', 'mias']

# Load Training and Test Data

In this section, the training/validation data is loaded. The load_data function pre-balances the data set by removing images from over-represented emotion classes.

In [5]:
metaData, meta2 = bc.load_training_metadata(dataPath, balanceViaRemoval=True, verbose=True)

Before Balancing
----------------
benign 63
malignant 1096
normal 3101
balanaceViaRemoval.avgE: 1420
balanaceViaRemoval.theshold: 1420.0

After Balancing
----------------
benign 63
malignant 859
normal 859


In [6]:
meta2[meta2.keys()[0]]

'0'

In [7]:
# Actually load some representative data for model experimentation
maxData = len(metaData)
X_data, Y_data = bc.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_data.shape
print Y_data.shape

Before Balancing
----------------
benign 63
malignant 1096
normal 3101
balanaceViaRemoval.avgE: 1420
balanaceViaRemoval.theshold: 1420.0

After Balancing
----------------
benign 63
malignant 859
normal 859
0.000000: A_0325_1.LEFT_MLO.LJPEG.png
0.028074: B_3073_1.RIGHT_MLO.LJPEG.png
0.056148: C_0221_1.RIGHT_MLO.LJPEG.png
0.084222: A_0146_1.RIGHT_MLO.LJPEG.png
0.112296: B_3031_1.RIGHT_MLO.LJPEG.png
0.140371: A_0152_1.LEFT_MLO.LJPEG.png
0.168445: B_3625_1.RIGHT_MLO.LJPEG.png
0.196519: B_3008_1.LEFT_MLO.LJPEG.png
0.224593: A_1050_1.RIGHT_MLO.LJPEG.png
0.252667: B_3013_1.LEFT_MLO.LJPEG.png
0.280741: B_3514_1.LEFT_CC.LJPEG.png
0.308815: A_0440_1.RIGHT_CC.LJPEG.png
0.336889: C_0093_1.LEFT_CC.LJPEG.png
0.364964: B_3405_1.LEFT_MLO.LJPEG.png
0.393038: B_3376_1.RIGHT_MLO.LJPEG.png
0.421112: A_0512_1.RIGHT_CC.LJPEG.png
0.449186: A_0434_1.RIGHT_CC.LJPEG.png
0.477260: A_0089_1.LEFT_MLO.LJPEG.png
0.505334: A_0139_1.LEFT_CC.LJPEG.png
0.533408: C_0220_1.RIGHT_CC.LJPEG.png
0.561482: A_0424_1.LEFT_CC.LJP

## Split Training/Test Sets
The following code segment splits the data into training and test data sets. Currently this is a standard 80/20 split for training and test respectively after performing a random shuffle using the unison_shuffled_copies help method.

In [8]:
skippedTransforms = True
if skippedTransforms:
    X_data2 = X_data
    Y_data2 = Y_data

In [9]:
# Split the data into Training and Test sets
trainNdx = int(X_data2.shape[0] * .8)
print trainNdx

X_train, X_test = np.split(X_data2, [trainNdx])
Y_train, Y_test = np.split(Y_data2, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

1311
(1311, 150, 150)
(328, 150, 150)
(1311, 1)
(328, 1)


In [10]:
import collections
def yDist(y):
    bcCounts = collections.defaultdict(int)
    for a in range(0, y.shape[0]):
        bcCounts[y[a][0]] += 1
    return bcCounts

print "Y_train Dist: " + str(yDist(Y_train))
print "Y_test Dist: " + str(yDist(Y_test))


Y_train Dist: defaultdict(<type 'int'>, {0: 661, 1: 50, 2: 600})
Y_test Dist: defaultdict(<type 'int'>, {0: 167, 1: 13, 2: 148})


## Define and Load Trained Model

In [11]:
# Load the bc array for our count in the model definition
bcTypes = bc.bcNumerics()
print bcTypes
print len(bcTypes)

{'benign': 1, 'malignant': 2, 'normal': 0}
3


In [12]:
# Construct the model using our help function
model = models.bc_model_v0(len(bcTypes), verbose=True, 
                                        input_shape=(1,X_train.shape[1],X_train.shape[2]))

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 32, 143, 143)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 67, 67)  25632       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

## Training the Model

The following code segment trains the model using the run_network helper function. 

In [13]:
loadWeights = False
if loadWeights:
    model.load_weights("dwdii-bc-150-v0-Cloud.hdf5")

In [14]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, X_train.shape[1],X_train.shape[2])
trainX = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],X_train.shape[2])

In [15]:
print "Training start: " + str(datetime.datetime.now())
m, h = models.run_network([trainX, testX, Y_train, Y_test], model, batch=50, epochs=30, verbosity=1)

Training start: 2017-03-22 03:28:50.565039
(1311, 3)
(328, 3)
Training model...
Train on 1311 samples, validate on 328 samples
Epoch 1/30
1311/1311 [==============================] - 352s - loss: 0.5424 - acc: 0.5812 - val_loss: 0.4755 - val_acc: 0.6250
Epoch 2/30
1311/1311 [==============================] - 319s - loss: 0.4671 - acc: 0.6323 - val_loss: 0.5179 - val_acc: 0.6280
Epoch 3/30
1311/1311 [==============================] - 339s - loss: 0.4542 - acc: 0.6468 - val_loss: 0.4581 - val_acc: 0.6250
Epoch 4/30
1311/1311 [==============================] - 309s - loss: 0.4463 - acc: 0.6568 - val_loss: 0.5710 - val_acc: 0.4909
Epoch 5/30
1311/1311 [==============================] - 307s - loss: 0.4328 - acc: 0.6613 - val_loss: 0.4686 - val_acc: 0.6402
Epoch 6/30
1311/1311 [==============================] - 302s - loss: 0.4132 - acc: 0.6773 - val_loss: 0.4457 - val_acc: 0.6616
Epoch 7/30
1311/1311 [==============================] - 303s - loss: 0.4007 - acc: 0.6941 - val_loss: 0.4941 - 

In [16]:
model.save_weights("dwdii-bc-150-v0-Cloud-3.hdf5", overwrite=True)

### Experiment Results

Initial results based on "normal" being masked as "benign":
* bc_model_v0 (150x150, 800/200): 182s - loss: 0.0560 - acc: 0.9813 - val_loss: 1.9918 - val_acc: 0.6800
* bc_model_v0 (150x150, 2000/500): 473s - loss: 0.0288 - acc: 0.9925 - val_loss: 1.4040 - val_acc: 0.7260
   * somewhat balanced, Y_train Dist {0: 1223, 1: 777}, Y_test Dist: {0: 321, 1: 179}

Revised with "normal", "benign" and "malignant" labeled seperately:
* bc_model_v0 (150x150, 1311/328): 298s - loss: 0.0411 - acc: 0.9786 - val_loss: 1.3713 - val_acc: 0.6616

In [17]:
661 / 1311.0

0.5041952707856598

In [18]:
167.0 / 328

0.5091463414634146